# 0. 🎯 Objectifs

1. Introduction aux Notebook Python
2. Introduction à RasterIO

***texte en gras***



```
# Ce texte est au format code
```



In [ ]:
print('test')

# 0. Vérification de l'envirronement et installation de librairies

In [ ]:
!nvidia-smi
!lscpu
!lsb_release -a

In [ ]:
%%capture --no-stderr
!pip3 -q install  rich typer[all] watermark rasterio
from rich import pretty, print, inspect, traceback
from rich.progress import track
pretty.install()
import logging
from rich.logging import RichHandler

FORMAT = "%(message)s"
logging.basicConfig(level= logging.INFO, format=FORMAT, datefmt="[%X]", handlers=[RichHandler(rich_tracebacks=True)],  # <- not sets rich_tracebacks
)
%load_ext watermark

Liste des commandes magiques: https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab

In [ ]:
%quickref

watermark est une commande magique pour vérifier l'installation des librairies

In [ ]:
%watermark?

In [ ]:
from watermark import watermark
print(watermark())

In [ ]:
%watermark -p gdal,rasterio,xarray,cv2

`%%` permet de changer la fonction de la cellule, par exemple pour passer à un shell:

In [ ]:
%%bash
ls -l
pwd

Le `!` permet d'exécuter des commandes Linux directement.

Installation de librairies Linux supplémentaires:

In [ ]:
!apt install tree

In [ ]:
!tree

# 1. Lecture des images
Les deux librairies principales pour lire des images sont:
* GDAL
* RasterIO (basé sur GDAL)

Téléchargement d'une image Ikonos:

In [ ]:
#https://drive.google.com/file/d/1_r4etnljaDbF42zNWp_3X0hL6xS5hzaH/view?usp=sharing
#import gdown
#gdown.download(
#        f"https://drive.google.com/uc?export=download&confirm=pbef&id=1_r4etnljaDbF42zNWp_3X0hL6xS5hzaH",
#        '/content/Boston.zip'
#    )
#!unzip -oq /content/Boston.zip -d /content
#img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'

Il y a la possibilité aussi de monter votre Google Drive:

In [ ]:
# This mounts your Google Drive to the Colab VM.
#from google.colab import drive
#drive.mount('/content/drive')


## 1.1 Commandes de base de GDAL

La librairie GDAL vient avec une collection de commandes:
https://gdal.org/programs/index.html


In [ ]:
!apt-get update
!apt-get install gdal-bin libgdal-dev

gdalinfo permet d'avoir de l'information sur une image

🔖 __ — via [gdalinfo — GDAL documentation](https://gdal.org/en/latest/programs/gdalinfo.html)

In [ ]:
!gdalinfo --help

On peut utiliser une variable python dans une commande avec le symbole $

In [ ]:
img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'
!gdalinfo $img_name # rajouter $ pour appeler une variable locale

On peut calculer des statistiques :

In [ ]:
!gdalinfo -stats -nogcp $img_name

Le *no data* permet de masquer les valeurs non valides.

On peut fixer la valeur de *no data* via la commande gdalwarp

🔖 __ — via [gdalwarp — GDAL documentation](https://gdal.org/en/latest/programs/gdalwarp.html)



In [ ]:
!gdalwarp -dstnodata 0 $img_name /content/output.tif

In [ ]:
!gdalinfo /content/output.tif
#img_name= '/content/output.tif'

In [ ]:
!gdalinfo -stats -nogcp /content/output.tif

GDAL vient aussi avec des scripts python:

In [ ]:
%run  /usr/bin/gdal_calc.py

Une opération courante est de changer de dynamique radiométrique, par exemple une onversion en 8 bit :

In [ ]:
#img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'
img_name = '/content/output.tif'
%run  /usr/bin/gdal_calc.py -A $img_name --outfile=/content/output_8bit.tif --allBands=A --overwrite --type=Byte --calc="(A-A.min())/(A.max()-A.min())*254+1" --NoDataValue=0
img_name = '/content/output_8bit.tif'

In [ ]:
!gdalinfo -stats -nogcp /content/output_8bit.tif

Il y a d'autres programmes très utiles comme [gdal_translate](https://gdal.org/programs/gdal_translate.html#gdal-translate)

### API Python de gdal (obsolète):

In [ ]:
import gdal, osr

ds = gdal.Open(img_name, gdal.GA_ReadOnly)
assert ds is not None, f"File not found: {img_name}"


Vous pouvez afficher les méthodes de l'objet:

In [ ]:
type(ds)

In [ ]:
dir(ds)

In [ ]:
inspect(ds, methods=True)

In [ ]:
inspect(ds, all=True)

On peut vérifier la taille de l'image:

In [ ]:
print("Size is {} x {} x {}".format(ds.RasterXSize,
                                    ds.RasterYSize,
                                    ds.RasterCount))

Obtenir la projection du raster et imprimer son code EPSG:

In [ ]:
prj=ds.GetProjection()
srs=osr.SpatialReference(wkt=prj)
epsg_code= srs.GetAttrValue('authority', 1)
print ("EPSG code: ", epsg_code)
print(f'https://epsg.io/?q={epsg_code}')

In [ ]:
## using GetGeoTransform we can get the upper left X and upper left y coordinates
ulx, xres, xskew, uly, yskew, yres  = ds.GetGeoTransform()

In [ ]:
 ## Calculate lower right x and lower right y we have the coordinates to build a polygon
## these values will be returned and be inputs into build bounds
lrx = ulx + (ds.RasterXSize * xres)
lry = uly + (ds.RasterYSize * yres)

In [ ]:
## rows and columns of the imagery (if needed)
cols = ds.RasterXSize
rows = ds.RasterYSize

In [ ]:
## print it all out to command line
print ("Number of columns: " + str(cols))
print ("Number of rows: " + str(rows))

In [ ]:
print ("___")
print ("upper left x, upper left y, lower right x, lower right y:")
print (ulx)
print (uly)
print (lrx)
print (lry)

In [ ]:
coords = [(ulx,lry), (ulx,uly), (lrx,uly), (lrx,lry)]
print("coords")
print(coords)

Conversion vers une matrice de type `numpy.ndarray`:

In [ ]:
import numpy as np
image = []
target_bands= [1,2,3] # Dans GDAL les bandes commencent à 1
for channel in target_bands:
    image_arr = ds.GetRasterBand(channel).ReadAsArray()
    nodatavalue= ds.GetRasterBand(channel).GetNoDataValue()
    assert image_arr is not None, f"Band not found: {channel}"
    image.append(image_arr)
image = np.dstack(image) # transform la liste de matrices en une matrice 3d
ds = None # c'est important de libérer le fichier une fois utilisé

In [ ]:
type(image)

In [ ]:
print(image.shape)
print(image.dtype)

## 1.2 RasterIO

RasterIO emballe GDAL et est plus facile à utiliser.

La librairie Rasterio offre de nombreuses fonctionnalités pour manipuler des données raster géospatiales en Python. Voici un aperçu de ses principales capacités :

### Lecture et écriture de données raster

- Lecture et écriture de nombreux formats de fichiers raster géospatiaux (GeoTIFF, NetCDF, JPEG2000, etc.)[1][2]
- Accès aux métadonnées des fichiers raster (système de coordonnées, emprise, résolution, etc.)[1]
- Lecture et écriture de bandes individuelles ou multiples[5]
- Support de la lecture/écriture par fenêtres (portions du raster)[5]

### Manipulation des données

- Conversion des données raster en tableaux NumPy pour faciliter les traitements[1][2]
- Rééchantillonnage et reprojection des données[1]
- Extraction de formes vectorielles à partir de données raster[3]
- Calcul de statistiques sur les données raster[4]

### Géoréférencement

- Gestion des systèmes de coordonnées et transformations[1]
- Conversion entre coordonnées image (lignes/colonnes) et coordonnées géographiques[1]

### Fonctionnalités avancées

- Support de la lecture/écriture de fichiers volumineux grâce à l'accès par fenêtres[5]
- Intégration avec d'autres bibliothèques scientifiques Python comme NumPy, SciPy, etc.[2]
- Interface en ligne de commande "rio" pour certaines opérations courantes[2]

### Performance

- Lecture/écriture rapide et efficace des données raster[2]
- Optimisations pour le traitement de grands volumes de données[5]

Rasterio s'appuie sur la bibliothèque GDAL pour la prise en charge des formats, tout en fournissant une API Python plus intuitive et plus facile à utiliser pour les développeurs Python[1][2]. Elle est largement utilisée dans le domaine de la géomatique et de la télédétection pour le traitement de données raster géospatiales.

**Références**:

[1] https://rasterio.readthedocs.io/en/stable/

[2] https://pypi.org/project/rasterio/1.3.0.post1/

[3] https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html

[4] https://briques-de-geomatique.readthedocs.io/fr/latest/logiciels-outils.html

[5] https://rasterio.readthedocs.io/en/stable/topics/reading.html

[6] https://briques-de-geomatique.readthedocs.io/fr/latest/format-donnees-import.html

[7] https://rasterio.readthedocs.io/en/stable/quickstart.html

[8] https://rasterio.readthedocs.io/en/stable/installation.html

In [ ]:
import rasterio
img_name = '/content/output_8bit.tif'
with rasterio.open(img_name) as src:
  print('CRS:',src.crs)
  print('WKT:',src.crs.to_wkt())
  meta = src.meta
  print('nodata: ',src.nodatavals)
  print('meta: ',meta)
  types= {i: dtype for i, dtype in zip(src.indexes, src.dtypes)}
  print(types)

`rio info` permet d'obtenir de l'information sur l'image:

In [ ]:
info = !rio info --indent 2 $img_name

In [ ]:
print('\n'.join(info))

In [ ]:
!rio info --indent 2 $img_name

Certains logiciels de traitement d'images en python organisent les tableaux différemment de rasterio. L'interprétation d'un tableau à 3 dimensions lu par rasterio est la suivante `(bands, rows, columns)`.

Tandis que les logiciels de traitement d'images comme `scikit-image`, `pillow` et matplotlib sont généralement commandés `(rows, columns, bands)`.

In [ ]:
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image
with rasterio.open(img_name) as src:
  raster = src.read()
  print('raster shape: ',raster.shape)

  image = reshape_as_image(raster)
  print('image shape: ',image.shape)

  raster2 = reshape_as_raster(image)
  print('raster shape: ',raster2.shape)

On peut visualiser avec matplotlib, observez le système de coordonnées:

In [ ]:
import matplotlib.pyplot as plt
plt.imshow?

In [ ]:
import rasterio
import matplotlib.pyplot as plt
with rasterio.open(img_name) as src:
  plt.imshow(src.read(1), cmap='pink')

In [ ]:

from rasterio.plot import show
with rasterio.open(img_name) as src:
  show(src)

Consultons l'histogramme des 3 bandes:

In [ ]:
from rasterio.plot import show_hist

img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'
#img_name = '/content/output_8bit.tif'
with rasterio.open(img_name) as src:
  show_hist(
      src, bins=50, lw=0.0, stacked=False, alpha=0.3,
      histtype='stepfilled', title="Histogram")

Vous aurez de la difficulté à visualiser en 16 bit:

In [ ]:
img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'
#img_name = '/content/output_8bit.tif'
fig, ax = plt.subplots(figsize=(27,27))
from rasterio.plot import show
with rasterio.open(img_name) as src:
  show(src.read([1,2,3]),transform=src.transform)

### Visualisation du masque no-data

In [ ]:
img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'
#img_name = '/content/output_8bit.tif'
with rasterio.open(img_name) as src:
  msk = src.read_masks(1)
  plt.imshow(msk)

### Visualisation et passage en 8bit avec numpy:

In [ ]:
img_name = '/content/054947080020_01/054947080020_01_P001_PSH/15SEP16155020-S2AS-054947080020_01_P001.TIF'
import numpy as np
import rasterio
import matplotlib.pyplot as plt
with rasterio.open(img_name) as src:
  img = src.read()
  print(img.shape)
  img = np.swapaxes(img, 0, 2) # il faut mettre la dimension des bandes en dernier
  print(img.shape)
  print(np.max(img))
  img = 1+254 * (img / float(np.max(img))) # scaling des valeurs entre 0 et 255
  plt.figure(figsize=(10,10))
  plt.imshow(img.astype(np.uint8))

# 2. Égalisation d'histogramme

L'égalisation d'histogramme est une technique importante en traitement d'image pour améliorer le contraste global d'une image. Voici les principes clés de cette méthode :

### Principe de base

L'égalisation d'histogramme vise à redistribuer les intensités des pixels d'une image de manière plus uniforme sur toute la plage de valeurs disponibles[1][6]. L'objectif est d'obtenir un histogramme plus équilibré, ce qui se traduit généralement par une amélioration du contraste de l'image.

### Étapes du processus

1. **Calcul de l'histogramme initial** : On détermine la distribution des intensités de pixels dans l'image d'origine.

2. **Calcul de l'histogramme cumulé** : On calcule l'histogramme cumulé, qui comptabilise pour chaque niveau de gris le nombre total de pixels ayant une intensité inférieure ou égale[4].

3. **Transformation des intensités** : On applique une fonction de transformation non linéaire basée sur l'histogramme cumulé pour modifier les intensités des pixels[1].

4. **Redistribution des intensités** : Les nouvelles intensités sont réparties sur toute la plage disponible, généralement de 0 à 255 pour une image en niveaux de gris 8 bits[1].

### Effets sur l'image

- **Amélioration du contraste** : L'égalisation d'histogramme augmente le contraste global de l'image, en particulier pour les images à faible contraste initial[8].

- **Étalement de la dynamique** : Les valeurs d'intensité sont étalées sur toute la plage disponible, ce qui peut faire ressortir des détails auparavant peu visibles[3].

- **Uniformisation de la distribution** : L'histogramme résultant tend vers une distribution plus uniforme, bien que parfaitement plate en pratique[2].

### Considérations importantes

- Cette technique est particulièrement efficace pour les images sombres ou à faible contraste.
- Elle peut parfois produire des résultats peu naturels ou exagérer le bruit dans certaines images.
- Pour les images couleur, l'égalisation est généralement appliquée sur la composante de luminance uniquement.

En résumé, l'égalisation d'histogramme est une méthode puissante pour améliorer automatiquement le contraste d'une image en redistribuant ses intensités de manière plus uniforme, ce qui peut révéler des détails auparavant difficiles à percevoir.

**Références**:

[1] https://fortierq.github.io/nb/hist_equal/

[2] https://perso.esiee.fr/~perretb/I5FM/TAI/histogramme/index.html

[3] http://bnazarian.free.fr/MyUploads/IN_GBM_03_PRE_TRAITEMENTS.PDF

[4] http://ressources.unit.eu/cours/videocommunication/UNIT_Image%20Processing_nantes/Version%20FR/Chapitre%202/Ressources/Transformation%20d'histogramme/Rchap2_TransfoHisto_FR%5Bfinal%5D.pdf

[5] http://www.normalesup.org/~pmaurel/IMA/CM/ima01_intro_imprimable.pdf

[6] https://fr.wikipedia.org/wiki/Histogramme_(imagerie_num%C3%A9rique)

[7] http://w4.uqo.ca/iglewski/ens/inf4173/2009h/Lariviere_final.pdf

[8] https://fastercapital.com/fr/contenu/Egalisation-de-l-histogramme---equilibre---amelioration-des-images-grace-a-l-egalisation-de-l-histogramme.html

![](https://upload.wikimedia.org/wikipedia/commons/c/ca/Histogrammeinebnung.png)

In [ ]:
def image_histogram_equalisation(image):
    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), 256, normed=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = (255-1) * cdf / cdf[-1] # normalize
    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)

    return image_equalized.reshape(image.shape).astype('uint8')


In [ ]:
img_he = image_histogram_equalisation(img[:,:,::-1])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_he)

In [ ]:
import numpy as np
import rasterio
from scipy.stats import norm

def gaussian_cdf(size=256):
    x = np.linspace(-3, 3, size)
    y = norm.cdf(x)
    return y

def cdf(im):
    """
    Computes the CDF of an image
    """
    hist, _ = np.histogram(im.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    return cdf / cdf[-1]

def hist_matching(c, c_t, im):
    """
    c: CDF of input image
    c_t: CDF of template (Gaussian in this case)
    im: input image
    """
    pixels = np.arange(256)
    new_pixels = np.interp(c, c_t, pixels)
    return new_pixels[im].astype(np.uint8)


# Generate Gaussian CDF
gaussian_cumulative = gaussian_cdf()

img_name = '/content/output_8bit.tif'
import numpy as np
import rasterio
import matplotlib.pyplot as plt
with rasterio.open(img_name) as src:
  image = src.read(1)
  profile = src.profile
  # Compute CDF of the input image
  image_cumulative = cdf(image)

  # Perform histogram matching
  matched = hist_matching(image_cumulative, gaussian_cumulative, image)

# Write the result
profile.update(dtype=rasterio.uint8, count=1)
with rasterio.open('output_image.tif', 'w', **profile) as dst:
    dst.write(matched, 1)

print("Histogram matching completed. Output saved as 'output_image.tif'")

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(matched)

# 3. K-means Classification

In [ ]:
from sklearn.cluster import KMeans
#import gdal
import numpy as np

In [ ]:
help(KMeans)

In [ ]:
img_he= img.reshape((img.shape[0]*img.shape[1],img.shape[2]))
print(img_he.shape)

In [ ]:
%timeit
km = KMeans(n_clusters=7)
km.fit(img_he)
km.predict(img_he)

In [ ]:
out_dat = km.labels_.reshape((img.shape[0], img.shape[1]))

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(out_dat)

In [ ]:
with rasterio.open(img_name) as src:
  print('CRS:',src.crs)
  with rasterio.open(
      '/content/new.tif',
      'w',
      driver='GTiff',
      height=out_dat.shape[0],
      width=out_dat.shape[1],
      count=1,
      dtype=out_dat.dtype,
      crs=src.crs,
      transform=src.transform,
  ) as dst_ds:
    dst_ds.write(out_dat, 1)

Sauvegarde du fichier:

In [ ]:
with gdal.Open(img_name, gdal.GA_ReadOnly) as gdst:
  driverTiff = gdal.GetDriverByName('GTiff')
  clfds = driverTiff.Create('/content/classified.tif', gdst.RasterXSize, gdst.RasterYSize, 1, gdal.GDT_Float32)
  clfds.SetGeoTransform(gdst.GetGeoTransform())
  clfds.SetProjection(gdst.GetProjection())
  clfds.GetRasterBand(1).SetNoDataValue(255)
  clfds.GetRasterBand(1).WriteArray(out_dat)
  clfds = None

In [ ]:
with rasterio.open('/content/classified.tif') as src:
  show(src)

In [ ]:
!wget https://github.com/sfoucher/TraitementImagesPythonVol1/raw/refs/heads/main/data/chapitre01/subset_1_of_S2A_MSIL2A_20240625T153941_N0510_R011_T18TYR_20240625T221903_resampled.tif

# Xarray
Xarray est une bibliothèque Python puissante conçue pour travailler avec des tableaux multidimensionnels étiquetés[1][3]. Voici les principales caractéristiques de Xarray :

### Fonctionnalités clés

- **Tableaux étiquetés** : Xarray introduit des étiquettes sous forme de dimensions, coordonnées et attributs sur des tableaux de type NumPy[3].

- **Structures de données** : Les deux structures de données principales sont DataArray (tableau étiqueté) et Dataset (collection de DataArrays partageant les mêmes coordonnées)[5].

- **Interopérabilité** : Xarray s'intègre bien avec l'écosystème scientifique Python, notamment NumPy, Pandas, Dask et Matplotlib[3].

- **Opérations avancées** : Elle offre des fonctionnalités pour l'analyse et la visualisation de données multidimensionnelles[3].

### Avantages

- **Intuitivité** : L'utilisation d'étiquettes rend le code plus lisible et moins sujet aux erreurs[3].

- **Flexibilité** : Xarray permet de sélectionner des données par étiquette plutôt que par position, facilitant la manipulation des données[5].

- **Vectorisation** : Les opérations mathématiques sont vectorisées sur plusieurs dimensions en se basant sur les noms des dimensions[3].

- **E/S efficace** : Xarray prend en charge la lecture et l'écriture de divers formats de données, notamment NetCDF, HDF, Zarr et GRIB[3].

### Utilisation

Xarray est particulièrement utile pour travailler avec des données scientifiques multidimensionnelles, comme des séries temporelles climatiques ou des images satellite[1]. Elle simplifie considérablement la manipulation, l'analyse et la visualisation de ces types de données complexes.

En résumé, Xarray est un outil puissant qui combine la flexibilité de Pandas avec la puissance de calcul de NumPy pour les tableaux multidimensionnels, offrant ainsi une solution efficace pour l'analyse de données scientifiques en Python.

Citations:
[1] https://docs.xarray.dev/en/stable/

[2] https://www.data-bird.co/blog/bibliotheque-python

[3] https://xarray.dev

[4] https://openclassrooms.com/fr/courses/7771531-decouvrez-les-librairies-python-pour-la-data-science/7857439-manipulez-le-data-frame

[5] https://docs.xarray.dev/en/stable/getting-started-guide/quick-overview.html

[6] https://docs.python.org/3.7/library/

[7] https://sist.pages.in2p3.fr/webinaire_netcdf_2024/python-xarray.html

[8] https://training.digitalearthafrica.org/fr/latest/python_basics/05_xarray.html


In [ ]:
!pip install rioxarray

In [ ]:
import rioxarray as xr

ds = xr.open_rasterio(img_name)
ds

In [ ]:
inspect(ds,methods=True)

In [ ]:
ds.band

In [ ]:
#with xr.set_options(display_style="html"):
display(ds)

In [ ]:
da= ds.band
#with xr.set_options(display_style="html"):
display(da)

In [ ]:
ds.dims

In [ ]:
ds.attrs

On peut sélectionner les données directement par *slicing* à la NumPy:

In [ ]:
ds.data[:30, 20:40]

On peut aussi utiliser la fonction `isel` sur le dataset:

In [ ]:
ds.isel(x=slice(None, 30), y=slice(20, 40))

Un autre façon de faire est d'utiliser les étiquettes des axes:

In [ ]:
out= ds.isel(band=1).sel(x=slice(4682000, 4682500,10), y=slice(330500,331000,10))
out

In [ ]:
ds.sel(band=2).sel(y=4686000, method="nearest").plot()

In [ ]:
ds.isel(band=1).plot.imshow()

In [ ]:
ds.plot.imshow(robust=True)